# Build NN model to predict TPSA from Pharmacophores

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [82]:
# Models
import os, sys
import math
import pickle
import itertools
import copy 
import joblib
import logging 
import types
# from multiprocessing import Pool, process
from datetime import datetime, time
from collections import defaultdict
from collections.abc import Iterator

if './src' not in sys.path:
    print(f"insert ./src")
    sys.path.insert(0, './src')
print(sys.path)

import numpy as np
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

import pandas as pd
pd.options.display.width = 170

import scipy.stats as sps 
import matplotlib.pyplot as plt
import torch
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

import warnings
warnings.filterwarnings('ignore')

from utils_cellpainting import *
from utils import display_gpu_info, display_gpu_device_info
from utils_ml import model_selection 

os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

setup_logging()
logging.info(f" 1/7- engine connected")
# logging.warning(f" 1/7- engine connected")
# logging.error(f" 1/7- engine connected")
# logging.critical(f" 1/7- engine connected")
# print(logging.NOTSET, logging.DEBUG,  logging.INFO, logging.WARN, logging.WARNING, logging.ERROR, logging.CRITICAL,  logging.FATAL)

print()
for time_fmt in ['%x %X', '%X %x %Z', '%X.%f', '%D-%X.%f', '%Y-%m-%d %H:%M:%S.%f']:
    cmd_string = f"datetime.now().strftime('{time_fmt}')"
    print(f" {cmd_string:50s}  : {datetime.now().strftime(time_fmt)}")

2024-04-24 12:31:28,241 - INFO: -  1/7- engine connected


['/tmp/dask-scratch-space/scheduler-tbhg9w8e', './src', '/home/kevin/WSL-shared/Cellpainting/cj-datasets', '/home/kevin/miniforge3/envs/cp311/lib/python311.zip', '/home/kevin/miniforge3/envs/cp311/lib/python3.11', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg']

 datetime.now().strftime('%x %X')                    : 04/24/24 12:31:28
 datetime.now().strftime('%X %x %Z')                 : 12:31:28 04/24/24 
 datetime.now().strftime('%X.%f')                    : 12:31:28.242589
 datetime.now().strftime('%D-%X.%f')                 : 04/24/24-12:31:28.242602
 datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')     : 2024-04-24 12:31:28.242614


In [83]:
import optuna
import dask
import dask.array as da
import dask.dataframe as dd

from dask import delayed
from dask.distributed import Client
from dask.distributed import LocalCluster
from dask_cuda import LocalCUDACluster
import dask_ml.model_selection as dcv

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import sklearn.metrics as skm

# from dask_ml.model_selection import train_test_split
# from dask_ml.model_selection import GridSearchCV, IncrementalSearchCV, HyperbandSearchCV
# from dask_ml.metrics import mean_squared_error, r2_score, mean_squared_log_error

from dask_ml.model_selection import train_test_split
from dask.distributed import Client
from dask.distributed import LocalCluster
from dask_cuda import LocalCUDACluster

import torch
from torch.utils.data import IterableDataset, DataLoader
print(torch.__version__)

2.2.0


In [4]:
# display_gpu_info()

In [84]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

# utility class to help normalize labels such that they contain only values between 0 and n_classes-1. 
from sklearn.preprocessing import LabelEncoder, Normalizer, normalize, MaxAbsScaler,MinMaxScaler

#K-Folds cross-validator - Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).
# Each fold is then used once as a validation while the k - 1 remaining folds form the training set.
from sklearn.model_selection import train_test_split, KFold

# Evaluate metric(s) by cross-validation and also record fit/score times.
from sklearn.model_selection import cross_validate, cross_val_score 

# Generate cross-validated estimates for each input data point.
# The data is split according to the cv parameter. Each sample belongs to exactly one test set, 
# and its prediction is computed with an estimator fitted on the corresponding training set.

from sklearn.model_selection import cross_val_predict

# Exhaustive search over specified parameter values for an estimator.
# Randomized search on hyper parameters.

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

#Input checker utility for building a cross-validator.(?)

from sklearn.model_selection._split import check_cv

from sklearn.pipeline import Pipeline

# Make a scorer from a performance metric or loss function. This factory function wraps scoring 
# functions for use in GridSearchCV and cross_val_score. It takes a score function, such as accuracy_score, 
# mean_squared_error, adjusted_rand_score or average_precision_score and returns a callable that 
# scores an estimator’s output. The signature of the call is (estimator, X, y) where estimator
# is the model to be evaluated, X is the data and y is the ground truth labeling (or None in the
# case of unsupervised models).

from sklearn.metrics import make_scorer

#regression matrics

from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score

#classification metrics

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin

# Construct a new unfitted estimator with the same parameters. Clone does a deep copy of the model 
# in an estimator without actually copying attached data. It returns a new estimator with the same 
# parameters that has not been fitted on any data.

from sklearn.base import clone


In [6]:
gzip_compression_options = {"method": "gzip", 'compresslevel': 1,"mtime": 1}

metadata_path = "./metadata/"
plates = pd.read_csv(os.path.join(metadata_path,"plate.csv"))
wells = pd.read_csv(os.path.join(metadata_path,"well.csv.gz"))
compound = pd.read_csv(os.path.join(metadata_path,"compound.csv.gz"))
orf = pd.read_csv(os.path.join(metadata_path,"orf.csv.gz"))


## gz, bz2, zip, tar, tar.gz, tar.bz2
# types = ['.gz', '.bz2','.zip', '.tar', '.tar.gz', '.tar.bz2']
type_bz2 = 'bz2'
type_gzip = 'gz'


## Datasets

In [7]:
prefix = '' ### Target-2' , 'MOA'

input_path ="./metadata/"
output_path ="./output_11102023/"

prefix_lc = prefix.lower().replace('-', '_')
CSV = '.csv'

# compoundMetadataInputFile   = f"{input_path}JUMP-{prefix}_compound_library.csv"

compoundMetadataInputFile          = f"{output_path}{prefix_lc}compound_metadata.csv"
compoundMetadataCleanFile          = f"{output_path}{prefix_lc}compound_metadata_clean.csv"
compoundMetadataTPSAFile           = f"{output_path}{prefix_lc}compound_metadata_tpsa.csv"
compoundMetadataTPSACleanFile      = f"{output_path}{prefix_lc}compound_metadata_tpsa_clean.csv"
compoundTPSAFile                   = f"{output_path}{prefix_lc}compound_TPSA.csv"
compoundTPSACleanFile              = f"{output_path}{prefix_lc}compound_TPSA_clean.csv"
compoundPharmacophoreFile          = f"{output_path}{prefix_lc}compound_pharmacophores_sparse.pkl"
compoundPharmacophoreCleanFile     = f"{output_path}{prefix_lc}compound_pharmacophores_sparse_clean.pkl"
compoundPharmacophoreDenseZipFile  = f"{output_path}{prefix_lc}compound_pharmacophores_dense.npz"
CompoundExtendedMetadataFile        = f"{output_path}{prefix_lc}compound_extended_metadata.csv"

# CompoundExtendedMetadataSampleFile  = f"{output_path}{prefix_lc}compound_extended_metadata_samples.csv"
CompoundExtendedMetadata5SampleFile = f"{output_path}{prefix_lc}compound_extended_metadata_5samples.csv"
CompoundExtendedMetadata3SampleFile = f"{output_path}{prefix_lc}compound_extended_metadata_3samples.csv"
CompoundExtendedMetadata2SampleFile = f"{output_path}{prefix_lc}compound_extended_metadata_2samples.csv"

# CompoundProfiles2SampleFileParquet  = f"{output_path}{prefix_lc}compound_profiles_2samples.parquet"
# CompoundProfiles2SampleFileGZ       = f"{output_path}{prefix_lc}compound_profiles_2samples.gz"

featureSelectionFile                = f"{output_path}{prefix_lc}normalized_feature_select.csv.gz"

# CompoundProfile5SampleInputFiles     = f"{output_path}/5sample_profiles/5sample_profiles_{{0:03d}}.csv"
CompoundProfile3SampleInputFiles     = f"{output_path}/3_sample_profiles/3sample_profiles_{{0:03d}}{{1:s}}.csv"
CompoundProfile2SampleInputFiles     = f"{output_path}/2_sample_profiles/2sample_profiles_{{0:03d}}{{1:s}}.csv"

# SampleProfiles                     = f"{output_path}/3sample_profiles/2sample_profiles_999_HashOrder.csv"
# 3SampleProfiles                     = f"{output_path}/3sample_profiles/2sample_profiles_999_HashOrder_{{0:03d}}.csv"
# binnedProfileFiles                  = f"{output_path}/2sample_profiles/binned_2sample_profiles_{{0:03d}}.csv"

trainingMetrics                     = f"{output_path}{prefix_lc}training_metrics.pkl"
profileMetadataFile                 = f"{input_path}profile_metadata.pkl"

In [8]:
print(f" ")
print(f" compound Metadata Input File             : {compoundMetadataInputFile}")
print(f" compound Metadata Output File            : {compoundMetadataCleanFile}\n")
print(f" compound Metadata + TPSA File            : {compoundMetadataTPSAFile}")
print(f" compound Metadata + TPSA Cleaned File    : {compoundMetadataTPSACleanFile}\n")
print(f" compound TPSA File                       : {compoundTPSAFile}")
print(f" compound TPSA Clean File                 : {compoundTPSACleanFile}")
print()
print(f" compound Pharmacophore Sparse File       : {compoundPharmacophoreFile}")
print(f" compound Pharmacophore Sparse Clean File : {compoundPharmacophoreCleanFile}")
print(f" compound Pharmacophore Dense Zipped File : {compoundPharmacophoreDenseZipFile}")
print(f" ")
print(f" Compound Extended MetadataFile           : {CompoundExtendedMetadataFile }")
print()
print(f" Compound Extended Metadata 5 SampleFile  : {CompoundExtendedMetadata5SampleFile}")
print(f" Compound Extended Metadata 3 SampleFile  : {CompoundExtendedMetadata3SampleFile}")
print(f" Compound Extended Metadata 2 SampleFile  : {CompoundExtendedMetadata2SampleFile}")
# print(f" ")
print(f" Compound Profile 3 Samples File          : {CompoundProfile3SampleInputFiles}")
print(f" Compound Profile 2 Samples File          : {CompoundProfile2SampleInputFiles}")
# print(f" CompoundProfiles2SamplesFile Parquet        : {CompoundProfiles2SampleFileParquet }")
# print(f" CompoundProfiles2SamplesFile gz             : {CompoundProfiles2SampleFileGZ }")
print(f" ")
print(f" featureSelectionFile                     : {featureSelectionFile}")

print()
print(f" profiles Metadata File                   : {profileMetadataFile}")
print(f" training metrics                         : {trainingMetrics}")
print()
# print(f" profileInputFiles                        : {profileInputFiles}")
# print(f" binnedProfileFiles_w_header              : {binnedProfileFiles_w_header}")
# print(f" binnedProfileFiles                       : {binnedProfileFiles}")

 
 compound Metadata Input File             : ./output_11102023/compound_metadata.csv
 compound Metadata Output File            : ./output_11102023/compound_metadata_clean.csv

 compound Metadata + TPSA File            : ./output_11102023/compound_metadata_tpsa.csv
 compound Metadata + TPSA Cleaned File    : ./output_11102023/compound_metadata_tpsa_clean.csv

 compound TPSA File                       : ./output_11102023/compound_TPSA.csv
 compound TPSA Clean File                 : ./output_11102023/compound_TPSA_clean.csv

 compound Pharmacophore Sparse File       : ./output_11102023/compound_pharmacophores_sparse.pkl
 compound Pharmacophore Sparse Clean File : ./output_11102023/compound_pharmacophores_sparse_clean.pkl
 compound Pharmacophore Dense Zipped File : ./output_11102023/compound_pharmacophores_dense.npz
 
 Compound Extended MetadataFile           : ./output_11102023/compound_extended_metadata.csv

 Compound Extended Metadata 5 SampleFile  : ./output_11102023/compound_extended

## Read Features metadata file

In [9]:
print(f" Features select file:  {profileMetadataFile}")

with open(profileMetadataFile, 'rb') as f: 
    pickle_data = pickle.load(f)

COMPOUND_PROFILE_COLUMNS = pickle_data['all_profile_columns']

metadata_categorical_columns = set(pickle_data['metadata_columns']['MetadataCategoricalColumns'])
metadata_categorical_columns_dtype = pickle_data['metadata_columns']['MetadataCategoricalColumns']

metadata_float_columns =  set(pickle_data['metadata_columns']['MetadataFloatColumns'])
metadata_float_columns_dtype =  pickle_data['metadata_columns']['MetadataFloatColumns']

metadata_integer_columns =  set(pickle_data['metadata_columns']['MetadataIntegerColumns'])
metadata_integer_columns_dtype =  pickle_data['metadata_columns']['MetadataIntegerColumns']

metadata_numeric_columns =  set(pickle_data['metadata_columns']['MetadataNumericColumns'])
metadata_numeric_columns_dtype =  pickle_data['metadata_columns']['MetadataNumericColumns']


MetadataColumns =  pickle_data['metadata_columns']['MetadataNames']
# MetadataNames   =  pickle_data['metadata_columns']['MetadataNames']

MetadataCategoricalColumns = pickle_data['metadata_columns']['MetadataCategoricalColumns']
MetadataFloatColumns       = pickle_data['metadata_columns']['MetadataFloatColumns']
MetadataIntegerColumns     = pickle_data['metadata_columns']['MetadataIntegerColumns']

ParquetColumns = pickle_data['parquet_columns']

X_columns = pickle_data['selected_columns']['selected']
X_columns_dtype = {x: np.float32 for x in X_columns}

y_columns = set(["Metadata_Permiation"])
y_columns_dtype = {x: np.int64 for x in y_columns} ## "Metadata_log10TPSA":np.dtype('float64')}

Xy_columns = list(y_columns | X_columns)
Xy_columns_dtype = y_columns_dtype | X_columns_dtype

 Features select file:  ./metadata/profile_metadata.pkl


In [59]:
disp_metadata_file(pickle_data, COMPOUND_PROFILE_COLUMNS)
print(f" len(X_columms)                ({len(X_columns):4d})")
print(f" len(X_columms_dtype)         : {len(X_columns_dtype)}")
print(f" len(y_columms)               : {len(y_columns):4d} {y_columns}")
print(f" len(y_columms_dtype)         : {len(y_columns_dtype)}   {y_columns_dtype}")
print(f" len(Xy_columms)               ({len(Xy_columns)}) {list(Xy_columns)[:3]}")
print(f" len(Xy_columms_dtype)         ({len(Xy_columns)}) {list(Xy_columns_dtype)[:3]}")

--------------------------------------------------------------------------------
 all_profile_columns & COMPOUND_PROFILE_COLUMNS
--------------------------------------------------------------------------------
  Len all_profile_columns        : 3672
  Len COMPOUND_PROFILE_COLUMNS   : 3672

  all_profile_columns[:10]       : ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA']
  COMPOUND_PROFILE_COLUMNS[:10]  : ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA']

  all_profile_columns[10:15]     : ['Metadata_Permiation', 'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea', 'Cells_AreaShape_BoundingBoxMaximum_X', 'Cells_AreaShape_BoundingBoxMaximum_Y']
  COMPOUND_PROFILE_COLUMNS[10:15]: ['Metadata_Permiation', 'Cells_AreaSh

In [26]:
# for k in pickle_data['metadata_columns'].keys():
#     print("-"*80)
#     print(f" {k}  - length({len(pickle_data['metadata_columns'][k])} )")
#     print("-"*80)
#     if isinstance(pickle_data['metadata_columns'][k], list):
#         for v in pickle_data['metadata_columns'][k]:
#             print(f" \t : list item : {v}")

#     elif isinstance(pickle_data['metadata_columns'][k], dict):    
#         for i,v in pickle_data['metadata_columns'][k].items():
#             print(f" \t : key :  {i:25s}     item: {v}")
#     print()

# print("\n all_profile_columns")
# print("-"*80)
# print(f" Length            : {len(COMPOUND_PROFILE_COLUMNS)}")
# print(f" profile cols [:5] : {COMPOUND_PROFILE_COLUMNS[:5]}")
# print(f" profile cols [:10]: {COMPOUND_PROFILE_COLUMNS[5:10]}")
# print(f" profile cols [:14]: {COMPOUND_PROFILE_COLUMNS[10:14]}")
# print(f" profile cols [:18]: {COMPOUND_PROFILE_COLUMNS[14:18]}")

# print()
# print("\n metadata_columns")
# print("-"*80)
# for i in pickle_data['metadata_columns'].keys():
#     print(f" {i:28s} ({len(pickle_data['metadata_columns'][i]):4d}) {list(pickle_data['metadata_columns'][i])[:5]}")
 
# print("\n selected_columns")
# print("-"*80)
# ttl = 0 
# for i in pickle_data['selected_columns'].keys():
#     ttl += len(pickle_data['selected_columns'][i])
#     print(f" Feature columns -/{i:25s}/   {len(pickle_data['selected_columns'][i]):5d}")
# print(f" {'total':47s}{ttl:5d}     ")     
 
# print('\n')
# print(f" metadata_categorical_columms  ({len(metadata_categorical_columns):4d}) {metadata_categorical_columns} ")
# print(f" metadata_float_columms        ({  len(metadata_float_columns):4d}) {metadata_float_columns}   ")
# print(f" metadata_integer_columms      ({len(metadata_integer_columns):4d}) {metadata_integer_columns} ")
# print(f" metadata_numeric_columms      ({len(metadata_numeric_columns):4d}) {metadata_numeric_columns} ")
# print(f" COMPOUND_PROFILE_COLUMNS      ({len(COMPOUND_PROFILE_COLUMNS):4d}) {COMPOUND_PROFILE_COLUMNS[:7]}")
# print()

print(f" len(X_columms)                ({len(X_columns):4d})")
print(f" len(X_columms_dtype)         : {len(X_columns_dtype)}")
print(f" len(y_columms)               : {len(y_columns):4d} {y_columns}")
print(f" len(y_columms_dtype)         : {len(y_columns_dtype)}   {y_columns_dtype}")
print(f" len(Xy_columms)               ({len(Xy_columns)}) {list(Xy_columns)[:3]}")
print(f" len(Xy_columms_dtype)         ({len(Xy_columns)}) {list(Xy_columns_dtype)[:3]}")

# print("\n Conversion Dictionaries")
# print("-"*80)
# print(f" categorical_columns_dtype     ({len(metadata_categorical_columns_dtype):4d}) {metadata_categorical_columns_dtype} ")
# print(f" float_columns_dtype           ({len(metadata_float_columns_dtype):4d}) {metadata_float_columns_dtype} ")
# print(f" integer_columns_dtype         ({len(metadata_integer_columns_dtype):4d}) {metadata_integer_columns_dtype} ")
# print('\n')



 len(X_columms)                (1471)
 len(X_columms_dtype)         : 1471
 len(y_columms)               :    1 {'Metadata_Permiation'}
 len(y_columms_dtype)         : 1   {'Metadata_Permiation': <class 'numpy.int64'>}
 len(Xy_columms)               (1472) ['Nuclei_Intensity_IntegratedIntensity_RNA', 'Nuclei_Granularity_5_Mito', 'Nuclei_Texture_Correlation_Mito_3_00_256']
 len(Xy_columms_dtype)         (1472) ['Metadata_Permiation', 'Nuclei_Intensity_IntegratedIntensity_RNA', 'Nuclei_Texture_Correlation_Mito_3_00_256']


In [12]:
parquetAllColumns = ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei'] | ParquetColumns['Image'] | ParquetColumns['Metadata']
parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']

print(f"\n\n")
print(f" Parquet file - all columns (Metadata + Cells + Cytoplasm + Nuclei + Images): {len(parquetAllColumns):7d}")
print(f"              - Image feature columns that are not considered, and dropped:   {len(ParquetColumns['Image']):7d}")
print(f"              - Parquet file metadata columns:                                {len(ParquetColumns['Metadata']):7d} \n")
print(f"              - all numeric columns (Cells + Cytoplasm + Nuclei):             {len(parquetNumericColumns):7d} \n")
 




 Parquet file - all columns (Metadata + Cells + Cytoplasm + Nuclei + Images):    4765
              - Image feature columns that are not considered, and dropped:      1089
              - Parquet file metadata columns:                                      3 

              - all numeric columns (Cells + Cytoplasm + Nuclei):                3673 



In [16]:
# pickle_data['selected_columns']['dropped_nans']

In [15]:
##-----------------------------------------------------------------------
## Remove numeric columns that contains bad values
##-----------------------------------------------------------------------
for key in ParquetColumns.keys():
    print(f"\n ParquetColumns['{key}'] - numeric_columns before removal of Nans Columns : {len(ParquetColumns[key]):4d}")
    for x in pickle_data['selected_columns']['dropped_nans']:
    
        if x in ParquetColumns[key] :
            del ParquetColumns[key][x]
            print(f" \t FOUND in ParquetColumns['{key}'], deleted -- {x:35s}")
        else:
            print(f"\t NOT FOUND in ParquetColumns['{key}'], skipped -- {x:35s}")
            pass
    print(f" ParquetColumns['{key}'] - numeric_columns after removal of Nans Columns  : {len(ParquetColumns[key]):4d} \n")


 ParquetColumns['Metadata'] - numeric_columns before removal of Nans Columns :    3
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cytoplasm_AreaShape_FormFactor     
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cells_Neighbors_SecondClosestDistance_Adjacent
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cells_Neighbors_FirstClosestDistance_5
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cells_Neighbors_AngleBetweenNeighbors_5
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cells_Neighbors_AngleBetweenNeighbors_Adjacent
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cells_Neighbors_SecondClosestDistance_5
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Nuclei_AreaShape_FormFactor        
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Cells_Neighbors_FirstClosestDistance_Adjacent
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Nuclei_Neighbors_FirstClosestDistance_1
	 NOT FOUND in ParquetColumns['Metadata'], skipped -- Nucl

In [17]:
parquetAllColumns = ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei'] | ParquetColumns['Image'] | ParquetColumns['Metadata']
parquetNumericColumns  =  ParquetColumns['Cells'] | ParquetColumns['Cytoplasm'] | ParquetColumns['Nuclei']

In [18]:
print(f"\n")
print(f" Parquet file - all columns (Metadata + Cells + Cytoplasm + Nuclei + Images): {len(parquetAllColumns):7d}")
print(f"              - Image feature columns that are not considered, and dropped:   {len(ParquetColumns['Image']):7d}")
print(f"              - Parquet file metadata columns:                                {len(ParquetColumns['Metadata']):7d} \n")
print(f"              - all numeric columns (Cells + Cytoplasm + Nuclei):             {len(parquetNumericColumns):7d} \n")
  



 Parquet file - all columns (Metadata + Cells + Cytoplasm + Nuclei + Images):    4753
              - Image feature columns that are not considered, and dropped:      1089
              - Parquet file metadata columns:                                      3 

              - all numeric columns (Cells + Cytoplasm + Nuclei):                3661 



In [19]:
##-----------------------------------------------------------------------
## Conversion dictionary for reading CSV files 
##-----------------------------------------------------------------------
profileConversionDict = dict()
## Add numeric features 
    
print(f"\n conversion dict length: {len(profileConversionDict)}")
## Add Metadata float columns to conversionDict:
profileConversionDict |= {x:y  for x,y in MetadataFloatColumns.items()}

## Add Integer metadata columns to conversion Dict
profileConversionDict |= {x:y  for x,y in MetadataIntegerColumns.items()}
    
## Add Categorical metadata columns to conversion Dict
profileConversionDict |= {x:y  for x,y in MetadataCategoricalColumns.items()}

# for k,v in parquetNumericColumns.items():
    # profileConversionDict[k] = np.float32
profileConversionDict |= {x: np.float32 for x in parquetNumericColumns.keys()} 

print(f" conversion dict length (after addition of Categorical columns): {len(profileConversionDict)}")

##-----------------------------------------------------------------------
## get_column list for fields to read from Parquet files 
##-----------------------------------------------------------------------
 
READ_PARQUET_COLUMNS = list(ParquetColumns['Metadata'] | parquetNumericColumns)
## join Columns 
JOIN_COLUMNS = MetadataColumns + list(parquetNumericColumns)

##
print()
print(f" Parquet Metadata Columns : {len(ParquetColumns['Metadata']):4d}  : {ParquetColumns['Metadata']}")
print(f" Parquet Numeric Columns  : {len(parquetNumericColumns):4d}")
print(f" Metadata + Selected Numeric Columns (READ_PARQUET_COLUMNS): {len(READ_PARQUET_COLUMNS):4d}  : {READ_PARQUET_COLUMNS[:5]} ...")
print()
print(f" JOIN_COLUMNS : join between parquet and compound extended metadata ")
print("-"*60)
print(f"    Metadata Columns              : {len(MetadataColumns):5d}")
print(f"    profile Numneric Columns      : {len(parquetNumericColumns):5d}")
print(f"    all Columns                   : {len(JOIN_COLUMNS):5d}")   
print('\n')
print(f" Numeric columns :")
print(f"------------------")
col_dict = cat_columns(parquetNumericColumns)


 conversion dict length: 0
 conversion dict length (after addition of Categorical columns): 3671

 Parquet Metadata Columns :    3  : {'Metadata_Source': dtype('O'), 'Metadata_Plate': dtype('O'), 'Metadata_Well': dtype('O')}
 Parquet Numeric Columns  : 3661
 Metadata + Selected Numeric Columns (READ_PARQUET_COLUMNS): 3664  : ['Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea'] ...

 JOIN_COLUMNS : join between parquet and compound extended metadata 
------------------------------------------------------------
    Metadata Columns              :    11
    profile Numneric Columns      :  3661
    all Columns                   :  3672


 Numeric columns :
------------------
    type: Cells                   count:  1249
    type: Cytoplasm               count:  1240
    type: Nuclei                  count:  1172
				  total:  3661


## Create dask cluster and client

In [30]:
!nvidia-smi

Wed Apr 24 10:57:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN Xp                Off |   00000000:09:00.0 Off |                  N/A |
| 23%   31C    P8              9W /  250W |       4MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [31]:


try:
    cluster.close()
    del cluster
except Exception as e:
    print("Cluster close failed")
else:
    print("Cluster close succeeded")
finally:
    print("Cluster finally clause")

Cluster close failed
Cluster finally clause


In [32]:
n_workers = 3
n_threads = 2
cuda_visible_devices = [0,1,2]
cluster = LocalCUDACluster( CUDA_VISIBLE_DEVICES=[0,1,2], n_workers=n_workers, threads_per_worker= n_threads, memory_limit='6GB', silence_logs = logging.WARNING)
# cluster = LocalCUDACluster( CUDA_VISIBLE_DEVICES=cuda_visible_devices, n_workers=n_workers, threads_per_worker= n_threads, silence_logs = logging.WARNING)
# cluster = LocalCluster("Kevins_Cluster", n_workers=2, threads_per_worker=2)
print(f" Cluster dashboard link : {cluster.dashboard_link}")

/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


 Cluster dashboard link : http://127.0.0.1:36589/status


In [34]:
# print(cluster)
# cluster.workers[2].restart
# cluster.start_worker
# cluster.worker_spec
# cluster.scale(3)

In [35]:
try:
    client.close()
    del client
except Exception as e:
    print("Client close failed")
else:
    print("Client close succeeded")
finally:
    print("Client finally clause")

Client close failed
Client finally clause


In [36]:
client = Client(cluster)    
print(f"Client created - {client.status}")

Client created - running


In [37]:
client  
# client = Client(cluster.scheduler_address)
# client = Client(processes = False)
# client = Client("tcp://127.0.0.1:37937")

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:36589/status,
Dashboard: http://127.0.0.1:36589/status,Workers: 3
Total threads: 6,Total memory: 16.76 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39471,Workers: 3
Dashboard: http://127.0.0.1:36589/status,Total threads: 6
Started: Just now,Total memory: 16.76 GiB
Comm: tcp://127.0.0.1:43129,Total threads: 2
Dashboard: http://127.0.0.1:38475/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:43401,


# Data Load and Prep

### Check input file

In [53]:
df_profiles =  pd.read_csv('./output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv', 
                        # names = COMPOUND_PROFILE_COLUMNS,
                        skiprows = 0,
                        nrows = 1
                        )
READ_COMPOUND_PROFILE_COLUMNS = df_profiles.columns.to_list()

In [54]:
len(READ_COMPOUND_PROFILE_COLUMNS)
len(COMPOUND_PROFILE_COLUMNS)
len(READ_PARQUET_COLUMNS)
len(JOIN_COLUMNS)
len(Xy_columns_dtype)

type(Xy_columns)
usecols = ['Metadata_JCP2022']+Xy_columns  
dtype = Xy_columns_dtype
print(dtype['Metadata_Permiation'])
type(usecols), len(usecols)
type(dtype)
type(dtype), len(dtype)
type(READ_COMPOUND_PROFILE_COLUMNS), len(READ_COMPOUND_PROFILE_COLUMNS)
READ_COMPOUND_PROFILE_COLUMNS[:14]
# for i in READ_COMPOUND_PROFILE_COLUMNS:
#     # if i in usecols:
#     if i in Xy_columns_dtype:
#         pass
#     else:
#         print(f" {i} is not in usecols")

1482

3672

3664

3672

1472

...

(list, 1473)

dict

(dict, 1472)

(list, 1482)

['Metadata_Source',
 'Metadata_Batch',
 'Metadata_Plate',
 'Metadata_Well',
 'Metadata_JCP2022',
 'Metadata_Hash',
 'Metadata_Bin',
 'Metadata_TPSA',
 'Metadata_lnTPSA',
 'Metadata_log10TPSA',
 'Metadata_Permiation',
 'Cells_AreaShape_BoundingBoxMaximum_X',
 'Cells_AreaShape_Center_X',
 'Cells_AreaShape_Center_Y']

### Experiment with pytorch IterableDataset

In [55]:
# del CustomIterableDataset, train_dataset, train_dataloader

In [38]:
class CustomIterableDataset(IterableDataset):

    def __init__(self, filename = None, names = None, usecols = None, dtype = None, iterator=False, sample_size = None, chunksize = None, start = None, end = None):
        
        #Store the filename in object's memory
        self.filename = filename
        self.names = names
        self.dtype = dtype
        self.usecols = usecols
        self.iterator = iterator
        # chunksize should be a mulitple of and sample_size
        if chunksize is None:
            self.chunksize = sample_size
        self.sample_size = sample_size
        self.start = start
        self.end = end
        self.numrows = end-start
        self.file_iterator =  pd.read_csv(self.filename, 
                                names = self.names,
                                header = 0,
                                skiprows = self.start,
                                nrows = self.numrows,
                                dtype = self.dtype,
                                usecols = self.usecols,
                                iterator = self.iterator,
                                chunksize = self.chunksize,
                                )
        #And that's it, we no longer need to store the contents in the memory

    def preprocess(self, text):
        ### Do something with data here
        print(f" Running preprocess data \n")
        text_pp = text.lower().strip()
        print(len(text_pp))
        ###
        return text_pp

    def line_mapper(self, line):
        print(f" Running line_mapper \n")
        #Splits the line into text and label and applies preprocessing to the text
        # data  = line.to_numpy()
        # text = self.preprocess(text)
        # print(f" compound: {compound.shape}  -  {data}")
        return line

    def __iter__(self):
        print(f"Running CustomIterable Dataset __iter__ function\n", flush = True)
        #Create an iterator
        # file_itr = open(self.filename)
        # df_ps = dd.read_csv(profile_file, header=header, names = names, usecols = usecols, dtype = dtype) 
        # Map each element using the line_mapper
        # self.mapped_itr = map(self.line_mapper, self.file_iterator)
        # print(type(file_itr))        
        # res = self.file_iterator.get_chunk(self.chunksize)
        return self.file_iterator

In [39]:
def worker_init_fn(worker_id):
    print(f"  Running Worker_init_fn  \n", flush = True)
    worker_info = torch.utils.data.get_worker_info()
    # print(worker_info)
    dataset = worker_info.dataset  # the dataset copy in this worker process
    overall_start = dataset.start
    overall_end = dataset.end
    # configure the dataset to only process the split workload
    # per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    per_worker_sample_set = (overall_end - overall_start) // (worker_info.num_workers * dataset.sample_size)
    per_worker = per_worker_sample_set * dataset.sample_size
    worker_id = worker_info.id
    dataset.start = overall_start + worker_id * per_worker
    dataset.end = min(dataset.start + per_worker, overall_end)
    print(f" worker id: {worker_info.id}  per_worker_sample_set: {per_worker_sample_set}   sample_size: {per_worker} \n", flush = True)
    print(f" worker id: {worker_info.id}  overall start: {overall_start}    overall_end: {overall_end}   sample_size: {dataset.sample_size} \n", flush = True)
    print(f" worker id: {worker_info.id}  dataset.start: {dataset.start}    dataset.end: {dataset.end} \n", flush = True)

def custom_collate(batch):
    # worker_info = torch.utils.data.get_worker_info()
    print(f" Running custom_collate - batch type: {type(batch)} len: {len(batch)} type: {type(batch[0])}  shape: {batch[0].shape} ", flush = True)
    batch_numpy = np.concatenate(batch)
    # print(f"  Running custom_collate: ")
    # print(f"  Batch type: {type(batch)}  Length Batch {len(batch)}  type element {type(batch[0])}  length element: {(batch[0].shape)}" )
    # print(f"  Batch_numpy.shape:   {batch_numpy.shape}")
    plates  = batch_numpy[:,:4]
    compounds = batch_numpy[:,4]
    cmphash = batch_numpy[:,5:7]
    labels = batch_numpy[:,10]
    data = batch_numpy[:, 11:].astype(np.float32)
    # print(f" plates: {plates.shape}  compounds: {compounds.shape}  hash: {cmphash.shape} labels: {labels.shape} data: {data.shape}")
    return plates,compounds, cmphash, labels, data
    # else:  # Fall back to `default_collate`
        # return default_collate(batch)

In [40]:
def display_cellpainting_batch(batch_id, batch):
    print("-"*80)
    print(f" Batch Id: {batch_id}   {type(batch)}  Rows returned {len(batch[0])}  ")
    print("-"*80)
    plates, compounds, cmphash, labels, data = batch
    print(f"{len(labels)}")
    for i in range(len(labels)):
        print(f" {i:3d} | {plates[i,0]:9s} | {compounds[i]:12s} | {cmphash[i,0]} | {cmphash[i,1]:2d} | {labels[i]:1d} | {data[i,:3]}")


### Creating the dataloader

In [41]:
# del train_dataloader, train_dataset, cellpainting

In [44]:
cellpainting_ds = dict()
cellpainting_ds['dataset_path'] = './output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv'
cellpainting_ds['sample_size'] = 3
cellpainting_ds['chunksize']   = None
cellpainting_ds['conversions'] = Xy_columns_dtype
cellpainting_ds['train_start'] = 0
cellpainting_ds['train_end']   = 120
cellpainting_ds['test_start']  = 300000
cellpainting_ds['test_end']    = cellpainting_ds['test_start'] + 120
cellpainting_ds['batch_size']  = 10

In [45]:
#Creating the iterable dataset object
train_dataset = CustomIterableDataset(cellpainting_ds['dataset_path'],
                                    # names = READ_COMPOUND_PROFILE_COLUMNS,
                                    # usecols = ['Metadata_JCP2022'] + Xy_columns, 
                                    dtype = cellpainting_ds['conversions'] ,
                                    iterator = True,
                                    sample_size = cellpainting_ds['sample_size'] ,
                                    start = cellpainting_ds['train_start'],
                                    end =  cellpainting_ds['train_end'])  ## 346542

In [46]:
train_dataloader = DataLoader(train_dataset, 
                              batch_size=10, 
                              num_workers = 0, 
                              collate_fn=custom_collate,
                              worker_init_fn = worker_init_fn ,
                              shuffle= False
                             )
# dataloader = DataLoader(dataset, batch_size=12, num_workers=0, worker_init_fn=worker_init_fn)
train_batch_id = 0

In [47]:
for train_batch in train_dataloader:
    display_cellpainting_batch(train_batch_id, train_batch)
    train_batch_id +=1
    if train_batch_id ==3 :
        break

Running CustomIterable Dataset __iter__ function

 Running custom_collate - batch type: <class 'list'> len: 10 type: <class 'pandas.core.frame.DataFrame'>  shape: (3, 1482) 
--------------------------------------------------------------------------------
 Batch Id: 0   <class 'tuple'>  Rows returned 30  
--------------------------------------------------------------------------------
30
   0 | source_1  | JCP2022_006020 | -9223347314827979542 | 10 | 0 | [-0.4223006  -0.41497695  0.29678357]
   1 | source_10 | JCP2022_006020 | -9223347314827979542 | 10 | 0 | [-0.63460433 -0.6232315  -1.6046009 ]
   2 | source_3  | JCP2022_006020 | -9223347314827979542 | 10 | 0 | [ 0.51076114  0.5562974  -0.22456942]
   3 | source_6  | JCP2022_058905 | -9223229703754907651 | 13 | 0 | [-1.8321388 -1.8294874  1.0576701]
   4 | source_3  | JCP2022_058905 | -9223229703754907651 | 13 | 0 | [-1.1679074 -1.2440039  0.6854775]
   5 | source_10 | JCP2022_058905 | -9223229703754907651 | 13 | 0 | [-0.36346462 -0.30

### Test dataset

In [50]:
try:
    del test_dataset, test_dataloader
except: 
    pass

In [52]:
#Creating the iterable dataset object
start_row = 300000
end_row   = start_row + 120
test_dataset = CustomIterableDataset(cellpainting_ds['dataset_path'],
                                    # names = READ_COMPOUND_PROFILE_COLUMNS,
                                    # usecols = ['Metadata_JCP2022'] + Xy_columns, 
                                    # dtype = Xy_columns_dtype,
                                    dtype = cellpainting_ds['conversions'] ,
                                    iterator = True,
                                    sample_size = cellpainting_ds['sample_size'] ,
                                    start = cellpainting_ds['train_start'],
                                    end =  cellpainting_ds['train_end'])  ## 346542                                     


In [53]:
test_dataloader = DataLoader(test_dataset, 
                             batch_size=12, 
                             num_workers = 0, 
                             collate_fn=custom_collate, 
                             worker_init_fn = worker_init_fn ,
                             shuffle= False
                             )
test_batch_id = 0

In [54]:
for test_batch in test_dataloader:
    display_cellpainting_batch(test_batch_id, test_batch)
    test_batch_id +=1
    if test_batch_id ==3 :
        break

Running CustomIterable Dataset __iter__ function

 Running custom_collate - batch type: <class 'list'> len: 12 type: <class 'pandas.core.frame.DataFrame'>  shape: (3, 1482) 
--------------------------------------------------------------------------------
 Batch Id: 0   <class 'tuple'>  Rows returned 36  
--------------------------------------------------------------------------------
36
   0 | source_1  | JCP2022_006020 | -9223347314827979542 | 10 | 0 | [-0.4223006  -0.41497695  0.29678357]
   1 | source_10 | JCP2022_006020 | -9223347314827979542 | 10 | 0 | [-0.63460433 -0.6232315  -1.6046009 ]
   2 | source_3  | JCP2022_006020 | -9223347314827979542 | 10 | 0 | [ 0.51076114  0.5562974  -0.22456942]
   3 | source_6  | JCP2022_058905 | -9223229703754907651 | 13 | 0 | [-1.8321388 -1.8294874  1.0576701]
   4 | source_3  | JCP2022_058905 | -9223229703754907651 | 13 | 0 | [-1.1679074 -1.2440039  0.6854775]
   5 | source_10 | JCP2022_058905 | -9223229703754907651 | 13 | 0 | [-0.36346462 -0.30

### Testing the Compound Profile file for `Nan`s

In [ ]:
dataset_iter = iter(dataset)

In [69]:
rows = next(dataset_iter)
rows

,Metadata_JCP2022,Metadata_Permiation,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_MajorAxisLength,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
9,JCP2022_038591,0,-0.004487,-0.081855,0.316238,2.068044,1.437638,0.294576,-2.093339,1.301392,...,0.919958,1.159541,0.540352,0.923058,1.803371,0.638770,1.548086,0.621993,-0.278182,0.397897
10,JCP2022_038591,0,-0.066505,-0.115482,-1.396676,1.483031,0.564793,-0.025421,-1.185505,0.968889,...,-0.052787,-0.409581,-0.700792,-0.429499,0.206291,0.057873,-0.158368,-0.268733,-0.576205,-0.501044
11,JCP2022_038591,0,-0.483601,-0.202347,-0.982798,-1.037483,-1.556019,0.584025,1.294679,-1.856521,...,-0.629579,-1.524363,-0.589253,-1.336803,-0.564902,-0.499227,-0.082996,0.313212,0.317085,-0.735549


In [66]:
df_header = pd.read_csv("./output_11102023/3_sample_profiles/binned_3sample_profiles_1482_header_info.csv")
df_header
COMPOUND_PROFILE_COLUMNS_1482 = df_header.columns.to_list()
len(COMPOUND_PROFILE_COLUMNS_1482)

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256


1482

In [67]:
file_reader =  pd.read_csv('./output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder.csv', 
                        names = COMPOUND_PROFILE_COLUMNS_1482,
                        header = 0,
                        skiprows = 0,
                        # nrows = self.numrows,
                        dtype = Xy_columns_dtype,
                        # usecols = Xy_columns,
                        iterator = True,
                        chunksize = 10,
                        )

In [68]:
row_num = 0 
na_rows = 0
bad_columns = set()

In [69]:
# for chunk in file_reader:
for batch in file_reader:
    for (row_id, row) in batch.iterrows():
        # row_id
        # num_array = row[[Xy_columns]]
        # row.Metadata_Permiation
        # row_check.astype(int)
        # break
        # row_check = pd.isna(row)
        # bad_columns_count = row_check.astype(int).sum()
        # if bad_columns_count > 0:
        #     na_rows += 1
        #     if  bad_columns_count < 1477:
        #         # print(f" INVALID | {row_id:6d} | {na_rows:4d} | {row_check.sum():4d} | ## {row.index[pd.isna(row)].to_list()}")
        #         # bad_columns |= set(row.index[pd.isna(row)])
        #         # row.to_frame().transpose()
        #         pass
        #     else:
        #         print(f" INVALID | {row_id:6d} | {na_rows:4d} | {row_check.sum():4d} | ('{row.Metadata_Source}', '{row.Metadata_Batch}', '{row.Metadata_Plate}')")
        #         # row.to_frame().transpose()
        # row_num += 1
        if row_num > 900:
            break


print(f" Rows read : {row_num}   invalid rows : {na_rows}")
print(bad_columns)

0

0

0

0

0

...

0

1

1

KeyboardInterrupt: 

2024-04-24 11:22:00,982 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/distributed/nanny.py", line 976, in run
    await worker.finished()
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/distributed/core.py", line 628, in finished
    await self._event_finished.wait()
  File "/home/kevin/miniforge3/envs/cp311/lib/python3.11/asyncio/locks.py", line 213, in wait
    await fut
asyncio.exceptions.CancelledError

During handling of the above exception, anot

In [ ]:
row_num = 0 
# for chunk in file_reader:
for batch in file_reader:
    type(batch)
    batch.shape
    for row in batch:
        if np.isnan(row).any():
            print(f"{row_num:7d} | {row:3d} | {batch[0][i]:15s}  |  {batch[1][i]}  | {batch[2][i].shape} |  {batch[2][i]}")
            print(' ---- INVALID -----')
        row_num += 1
    break


## Other methods tried

### expriment with pytorch `Dataset`

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, IterableDataset, Dataset

# Breakdown file
# raw_ddf = dd.read_parquet("data.parquet") # Read huge file using dask
# raw_ddf = raw_ddf.set_index("userid") # set the userid as index
# userids = raw_ddf.index.unique().compute().values.tolist() # get a list of indices
# new_ddf = raw_ddf.repartition(divisions = userids) # repartition by userids
# new_ddf.to_parquet("my_folder") # this will save each user as its own parquet file within "my_folder"

# Dask to read the partitions
train_ddf = dd.read_csv("./output_11102023/3_sample_profiles/3sample_profiles_999_HashOrder.csv") # read all files

# Read labels file
# labels_df = pd.read("label.csv")
# y_labels = np.array(labels_df["class"])

# Define the Dataset class
class UsersDataset(Dataset):
    def __init__(self, dask_df, partition_map,  labels):
        self.dask_df = dask_df
        self.label_columns = labels
        self.partition_map = partition_map

    def __len__(self):
        return self.partition_map.sum()

    def __getitem__(self, idx): 
        row = self.dask_df.loc[idx].to_dict()
        tensor = torch.tensor(list(row.values()))
        
        # X_df = self.dask_df.get_partition(idx).compute()
        # X = np.row_stack([X_df])
        # X_tensor = torch.tensor(X, dtype=torch.float32)
        # y = self.labels[idx]
        # y_tensor = torch.tensor(y, dtype=torch.long)
        # sample = (X_tensor, y_tensor) 
        sample = tensor 
        return sample

In [54]:
train_ddf.npartitions
train_ddf.partitions[0].head()

383

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,-0.584637,-0.525943,-0.602353,-0.592855,-0.583981,-0.591326,-0.601436,-0.585525,-0.585781,-0.574300
1,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,0.145055,0.132206,0.133773,0.112987,0.107752,0.121523,0.135930,0.124801,0.113462,0.122250
2,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,2,71.18,4.265212,1.852358,...,0.758732,0.994908,0.930062,0.893926,0.927600,0.894280,0.861851,0.843516,0.854878,0.804462
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,1,58.65,4.071588,1.768268,...,-0.382233,-0.385646,-0.376897,-0.372728,-0.349244,-0.354360,-0.367956,-0.369715,-0.356706,-0.363440
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,1,58.65,4.071588,1.768268,...,-0.326972,-0.306617,-0.344224,-0.340425,-0.352327,-0.350739,-0.337809,-0.340152,-0.350334,-0.345115


In [72]:
partition_map = train_ddf.map_partitions(len).compute()

In [74]:
# dd.sum()
type(partition_map)

pandas.core.series.Series

In [80]:
# Create a Dataset object
user_dataset = UsersDataset(dask_df=train_ddf, partition_map = partition_map , labels = y_columns) 

In [81]:
user_dataset.dask_df

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,Metadata_Permiation,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EquivalentDiameter,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_MajorAxisLength,Cells_AreaShape_MaxFeretDiameter,Cells_AreaShape_MaximumRadius,Cells_AreaShape_MeanRadius,Cells_AreaShape_MedianRadius,Cells_AreaShape_MinFeretDiameter,Cells_AreaShape_MinorAxisLength,Cells_AreaShape_Orientation,Cells_AreaShape_Perimeter,Cells_AreaShape_Solidity,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_3_3,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_4_2,Cells_AreaShape_Zernike_4_4,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_5_3,Cells_AreaShape_Zernike_5_5,Cells_AreaShape_Zernike_6_0,Cells_AreaShape_Zernike_6_2,Cells_AreaShape_Zernike_6_4,Cells_AreaShape_Zernike_6_6,Cells_AreaShape_Zernike_7_1,Cells_AreaShape_Zernike_7_3,Cells_AreaShape_Zernike_7_5,Cells_AreaShape_Zernike_7_7,Cells_AreaShape_Zernike_8_0,Cells_AreaShape_Zernike_8_2,Cells_AreaShape_Zernike_8_4,Cells_AreaShape_Zernike_8_6,Cells_AreaShape_Zernike_8_8,Cells_AreaShape_Zernike_9_1,Cells_AreaShape_Zernike_9_3,Cells_AreaShape_Zernike_9_5,Cells_AreaShape_Zernike_9_7,Cells_AreaShape_Zernike_9_9,Cells_Children_Cytoplasm_Count,Cells_Correlation_Correlation_AGP_DNA,Cells_Correlation_Correlation_AGP_ER,Cells_Correlation_Correlation_AGP_Mito,Cells_Correlation_Correlation_AGP_RNA,Cells_Correlation_Correlation_DNA_ER,Cells_Correlation_Correlation_DNA_Mito,Cells_Correlation_Correlation_DNA_RNA,Cells_Correlation_Correlation_ER_Mito,Cells_Correlation_Correlation_ER_RNA,Cells_Correlation_Correlation_Mito_RNA,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_AGP_ER,Cells_Correlation_K_AGP_Mito,Cells_Correlation_K_AGP_RNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_DNA_RNA,Cells_Correlation_K_ER_AGP,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_ER_Mito,Cells_Correlation_K_ER_RNA,Cells_Correlation_K_Mito_AGP,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_Mito_ER,Cells_Correlation_K_Mito_RNA,Cells_Correlation_K_RNA_AGP,Cells_Correlation_K_RNA_DNA,Cells_Correlation_K_RNA_ER,Cells_Correlation_K_RNA_Mito,Cells_Correlation_Manders_AGP_DNA,Cells_Correlation_Manders_AGP_ER,Cells_Correlation_Manders_AGP_Mito,Cells_Correlation_Manders_AGP_RNA,Cells_Correlation_Manders_DNA_AGP,Cells_Correlation_Manders_DNA_ER,Cells_Correlation_Manders_DNA_Mito,Cells_Correlation_Manders_DNA_RNA,Cells_Correlation_Manders_ER_AGP,Cells_Correlation_Manders_ER_DNA,Cells_Correlation_Manders_ER_Mito,Cells_Correlation_Manders_ER_RNA,Cells_Correlation_Manders_Mito_AGP,Cells_Correlation_Manders_Mito_DNA,Cells_Correlation_Manders_Mito_ER,Cells_Correlation_Manders_Mito_RNA,Cells_Correlation_Manders_RNA_AGP,Cells_Correlation_Manders_RNA_DNA,Cells_Correlation_Manders_RNA_ER,Cells_Correlation_Manders_RNA_Mito,Cells_Correlation_Overlap_AGP_DNA,Cells_Correlation_Overlap_AGP_ER,Cells_Correlation_Overlap_AGP_Mito,Cells_Correlation_Overlap_AGP_RNA,Cells_Correlation_Overlap_DNA_ER,Cells_Correlation_Overlap_DNA_Mito,Cells_Correlation_Overlap_DNA_RNA,Cells_Correlation_Overlap_ER_Mito,Cells_Correlation_Overlap_ER_RNA,Cells_Correlation_Overlap_Mito_RNA,Cells_Correlation_RWC_AGP_DNA,Cells_Correlation_RWC_AGP_ER,Cells_Correlation_RWC_AGP_Mito,Cells_Correlation_RWC_AGP_RNA,Cells_Correlation_RWC_DNA_AGP,Cells_Correlation_RWC_DNA_ER,Cells_Correlation_RWC_DNA_Mito,Cells_Correlation_RWC_DNA_RNA,Cells_Correlation_RWC_ER_AGP,Cells_Correlation_RWC_ER_DNA,Cells_Co

In [116]:
user_dataset.label_columns
len(user_dataset)
len(user_dataset) /3

{'Metadata_Permiation'}

346542

115514.0

In [85]:
# Create a DataLoader object
dataloader = DataLoader(user_dataset, batch_size=4, shuffle=True, num_workers=0)

In [34]:
# Print output of the first batch to ensure it works
# for i_batch, sample_batched in enumerate(dataloader): 
#     print("Batch number ", i_batch)
#     print(sample_batched[0]) # print X
#     print(sample_batched[1]) # print y
#     print(sample_batched[2]) # print y
#     print(sample_batched[3]) # print y

#     # stop after first batch.
#     if i_batch == 0:
#         break

In [33]:
# user_dataset.dask_df.loc[12].compute()

### Experiment with pytorch `read_binned_profile_files`

In [58]:
# df_profiles = read_cell_profiles(input_files[i], names = COMPOUND_PROFILE_COLUMNS, usecols = Xy_columns, dtype = Xy_columns_dtype)
# df_profiles = read_cell_profiles(training_files, names = COMPOUND_PROFILE_COLUMNS, usecols = Xy_columns, dtype = Xy_columns_dtype)
# file_idxs = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

TRAINING_FILES = {0, 1, 2, 4, 5, 6, 7, 8, 9, 11}
TEST_FILES = {3, 10}
input_file_list = read_binned_profile_files(file_idxs = TRAINING_FILES, 
                                            filename = binnedProfileFiles,
                                            names = COMPOUND_PROFILE_COLUMNS, 
                                            usecols = Xy_columns, 
                                            dtype = Xy_columns_dtype)

2024-02-23 14:53:06,030 - INFO: -  Read profiles file ...


 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_000.csv
 Reading ALL  rows - Number of partitions:  21   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_001.csv
 Reading ALL  rows - Number of partitions:  20   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_002.csv
 Reading ALL  rows - Number of partitions:  21   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_004.csv
 Reading ALL  rows - Number of partitions:  21   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_005.csv
 Reading ALL  rows - Number of partitions:  21   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_006.csv
 Reading ALL  rows - Number of partitions:  21   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_007.csv
 Reading ALL  r

2024-02-23 14:53:08,143 - INFO: -  Read profiles file ... complete


 Reading ALL  rows - Number of partitions:  21   

 Reading cell profiles file :  ./output_11102023/binned_profiles/binned_2sample_profiles_011.csv
 Reading ALL  rows - Number of partitions:  21   



In [59]:
type(input_file_list)
len(input_file_list[0])
# input_file_list[0].head()
input_file_list[0].npartitions


list

19279

21

In [60]:
training_file = dd.concat(input_file_list)

In [33]:
# type(training_file)
# # training_file.head(5)
# training_file.loc[0].compute()


### expriment with NVTabular

In [34]:
import glob

In [23]:
from nvtabular.loader.torch import TorchAsyncItr, DLDataLoadera

2024-02-23 15:24:38,688 - INFO: - init


In [26]:
TRAIN_PATHS = glob.glob("./output_11102023/binned_profiles/*.csv")

In [31]:
TRAIN_PATHS

['./output_11102023/binned_profiles/binned_2sample_profiles_009.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_000.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_002.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_006.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_004.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_001.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_007.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_011.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_003.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_005.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_008.csv']

In [32]:
# TRAIN_PATHS.pop(7)

'./output_11102023/binned_profiles/binned_2sample_profiles_011.csv'

In [44]:
import nvtabular as nvt

In [45]:
train_dataset = TorchAsyncItr(
   nvt.Dataset(TRAIN_PATHS),
   cats=[],
   conts=X_columns,
   labels=y_columns,
   batch_size=16
)

/home/kevin/miniforge3/envs/cp311/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
2024-02-23 15:39:28,950 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.92 GiB -- Worker memory limit: 5.59 GiB
2024-02-23 15:39:38,951 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.46 GiB -- Worker memory limit: 5.59 GiB
2024-02-23 15:39:39,241 - distributed.worker.memory - WARNING - Worker i

In [ ]:
# input_file_list = read_binned_profile_files(file_idxs = TRAINING_FILES, 
#                                             filename = binnedProfileFiles,
#                                             names = COMPOUND_PROFILE_COLUMNS, 
#                                             usecols = Xy_columns, 
#                                             dtype = Xy_columns_dtype)

In [47]:
train_loader = DLDataLoader(
   train_dataset,
   batch_size=None,
   pin_memory=False,
   num_workers=0
)

In [48]:
for data in train_loader:
    len(data)
    # print(data[0])
    break

BufferError: DLPack only supports signed/unsigned integers, float and complex dtypes.

### Read compound profiles

In [90]:
# Apply feature selection
# for x in [9,10,11]:
#     readFile = binnedProfileFiles_w_header.format(x)
#     writeFile = binnedProfileFiles.format(x)
#     # profilesFile = CompoundProfiles2SampleFileCSV ## +'.'+ type_bz2
#     # featureSelectionFile = './output_11102023/normalized_feature_select.csv'
    
#     print(f" Profiles file to read       :  {readFile}")
#     print(f" Profiles file to Write      :  {writeFile}")
#     print(f" Features select file        :  {featureSelectionFile}")
    
#     # df_profiles = dd.read_csv(readFile, blocksize="48MB", header = 0 )  ##, index_col = 'CASRN')
#     df_profiles = pd.read_csv(readFile)  ##, index_col = 'CASRN')
#     df_profiles.info()
#     df_profiles.head(6)
    
#     print(f" Profiles file to Write      :  {writeFile}")
    
#     df_profiles.to_csv(writeFile, index = False, header = None)
    
#     df_profiles = pd.read_csv(writeFile, nrows = 10)  ##, index_col = 'CASRN')
#     df_profiles.info()
#     df_profiles.head(6)
#     print(f"  Write      :  {writeFile}  Complete")

In [63]:
# df_y = df_profiles[y_columns]
# df_X = df_profiles[list(x['selected'])] ## .drop(labels=x_columns_drop, axis =1)

In [91]:
# df_y.info()
# df_y.head()

df_profiles = pd.read_csv(writeFile, names = COMPOUND_PROFILE_COLUMNS,  )  ##, index_col = 'CASRN')
df_profiles.info()
df_profiles.head(6)
print(f"  Write      :  {writeFile}  Complete")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19358 entries, 0 to 19357
Columns: 3681 entries, Metadata_Source to Nuclei_Texture_Variance_RNA_5_03_256
dtypes: float64(3673), int64(3), object(5)
memory usage: 543.6+ MB


,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,source_1,Batch1_20221004,UL001641,Z22,JCP2022_000137,-9070688483616626497,11,60.28,4.099000,1.780173,...,-0.269112,-0.323952,-0.276385,-0.272236,-0.268422,-0.274739,-0.281269,-0.267840,-0.268723,-0.285727
1,source_1,Batch1_20221004,UL001641,E35,JCP2022_000727,6430107330365911187,11,41.90,3.735286,1.622214,...,-0.114117,-0.062046,-0.138007,-0.144709,-0.133281,-0.127678,-0.149397,-0.150446,-0.140857,-0.124807
2,source_1,Batch1_20221004,UL001641,AE32,JCP2022_000940,3151155857995480151,11,81.63,4.402197,1.911850,...,-0.231102,-0.260289,-0.186766,-0.213832,-0.231651,-0.234182,-0.204242,-0.233468,-0.238366,-0.257642
3,source_1,Batch1_20221004,UL001641,S26,JCP2022_004615,8072843849794829003,11,71.25,4.266195,1.852785,...,0.004440,0.010617,0.009598,0.018390,0.009552,0.018826,0.021466,0.019485,0.021801,0.032578
4,source_1,Batch1_20221004,UL001641,AE23,JCP2022_004722,-738460285612537225,11,139.77,4.939998,2.145414,...,-0.144550,-0.154846,-0.117254,-0.120452,-0.123090,-0.111855,-0.116635,-0.124112,-0.118747,-0.126915
5,source_1,Batch1_20221004,UL001641,AE35,JCP2022_006602,1103075696153509931,11,66.54,4.197803,1.823083,...,3.278914,3.296048,3.337265,3.371311,3.322510,3.290074,3.291197,3.277430,3.303241,3.273094


  Write      :  ./output_11102023/binned_profiles/binned_2sample_profiles_011.csv  Complete


In [92]:
df_profiles.loc[4:6]

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
4,source_1,Batch1_20221004,UL001641,AE23,JCP2022_004722,-738460285612537225,11,139.77,4.939998,2.145414,...,-0.144550,-0.154846,-0.117254,-0.120452,-0.123090,-0.111855,-0.116635,-0.124112,-0.118747,-0.126915
5,source_1,Batch1_20221004,UL001641,AE35,JCP2022_006602,1103075696153509931,11,66.54,4.197803,1.823083,...,3.278914,3.296048,3.337265,3.371311,3.322510,3.290074,3.291197,3.277430,3.303241,3.273094
6,source_1,Batch1_20221004,UL001641,D41,JCP2022_008889,-8951101999396403617,11,26.30,3.269569,1.419956,...,1.369933,1.213983,1.388841,1.408804,1.408017,1.381786,1.363386,1.378481,1.402009,1.317862


In [93]:
df_profiles.loc[240:260]

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
240,source_1,Batch1_20221004,UL001653,H05,JCP2022_045064,1682031449582562611,11,62.46,4.134526,1.795602,...,-0.227488,-0.186587,-0.250205,-0.242628,-0.242687,-0.225250,-0.242337,-0.233774,-0.241946,-0.216887
241,source_1,Batch1_20221004,UL001653,V33,JCP2022_048431,5684006070195228395,11,71.26,4.266335,1.852846,...,-0.082917,-0.066162,-0.083134,-0.090991,-0.109716,-0.092288,-0.059001,-0.098549,-0.101214,-0.075736
242,source_1,Batch1_20221004,UL001653,B25,JCP2022_049541,-7092836564153608321,11,70.99,4.262539,1.851197,...,-0.844099,-0.822597,-0.899051,-0.888266,-0.886928,-0.889793,-0.893467,-0.856428,-0.878572,-0.878533
243,source_1,Batch1_20221004,UL001653,AB18,JCP2022_058138,-7635582157158183913,11,50.94,3.930648,1.707059,...,0.266770,0.318888,0.350936,0.374537,0.389441,0.374583,0.355311,0.337836,0.362661,0.358314
244,source_1,Batch1_20221004,UL001653,U24,JCP2022_058868,4344739935189359615,11,54.71,4.002047,1.738067,...,0.414980,0.434439,0.445084,0.489713,0.467641,0.450078,0.443262,0.441577,0.467422,0.461985
245,source_1,Batch1_20221004,UL001653,C24,JCP2022_064272,-5922428777075812681,11,88.58,4.483906,1.947336,...,-0.271733,-0.199112,-0.274583,-0.257481,-0.277112,-0.274132,-0.268998,-0.252819,-0.284053,-0.269976
246,source_1,Batch1_20221004,UL001653,T30,JCP2022_064563,6249806720197255427,11,103.85,4.642948,2.016407,...,1.431586,1.362539,1.479455,1.428651,1.385897,1.409843,1.467172,1.376540,1.405258,1.401443
247,source_1,Batch1_20221004,UL001653,AC14,JCP2022_067927,6574466583603819095,11,61.42,4.117736,1.788310,...,-0.685992,-0.660406,-0.720947,-0.714537,-0.720276,-0.702047,-0.716185,-0.695294,-0.695200,-0.675941
248,source_1,Batch1_20221004,UL001653,L27,JCP2022_074006,-1587860076688117297,11,81.23,4.397285,1.909716,...,-0.371671,-0.412269,-0.378608,-0.365525,-0.377729,-0.373236,-0.380105,-0.373644,-0.373284,-0.367121
249,source_1,Batch1_20221004,UL001653,AA07,JCP2022_075997,-2806265087415870253,11,79.95,4.381401,1.902818,...,0.223884,0.175052,0.242497,0.233195,0.244996,0.242264,0.231883,0.218301,0.229454,0.213717


In [94]:
del df_profiles

### using DASK 

In [108]:
import glob

TRAIN_PATHS = glob.glob("./output_11102023/binned_profiles/binned_2sample_profiles_0*.csv")

TRAIN_PATHS.pop(7)
TRAIN_PATHS.pop(7)
TRAIN_PATHS

'./output_11102023/binned_profiles/binned_2sample_profiles_010.csv'

'./output_11102023/binned_profiles/binned_2sample_profiles_011.csv'

['./output_11102023/binned_profiles/binned_2sample_profiles_009.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_000.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_002.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_006.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_004.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_001.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_007.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_003.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_005.csv',
 './output_11102023/binned_profiles/binned_2sample_profiles_008.csv']

In [ ]:
names = COMPOUND_PROFILE_COLUMNS, 

In [109]:
df_ps = dd.read_csv(TRAIN_PATHS, header=None, names = COMPOUND_PROFILE_COLUMNS, usecols = Xy_columns, dtype = Xy_columns_dtype)   

In [110]:
df_ps.head()

,Metadata_Permiation,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_MajorAxisLength,Cells_AreaShape_MedianRadius,...,Nuclei_Texture_SumAverage_DNA_10_01_256,Nuclei_Texture_SumAverage_ER_10_01_256,Nuclei_Texture_SumAverage_Mito_10_01_256,Nuclei_Texture_SumAverage_RNA_10_01_256,Nuclei_Texture_SumEntropy_DNA_10_03_256,Nuclei_Texture_SumVariance_AGP_10_03_256,Nuclei_Texture_SumVariance_DNA_10_03_256,Nuclei_Texture_SumVariance_ER_10_01_256,Nuclei_Texture_SumVariance_Mito_10_03_256,Nuclei_Texture_SumVariance_RNA_10_01_256
0,0,2.031038,2.148220,-0.417483,0.028970,1.325444,0.564229,-1.144520,-0.205987,-0.571096,...,0.142510,-0.054428,0.222660,0.713571,-0.171428,-0.017424,-0.113027,-0.411903,-0.115996,-0.215810
1,0,0.028491,0.075577,-0.178618,-0.216195,0.110587,0.685035,-0.003664,-0.163900,-0.154681,...,-0.128096,-0.996086,-0.671036,-0.866938,0.368866,-0.593843,-0.176098,-0.673963,-0.451727,-0.416280
2,0,-1.039744,-1.031732,1.966256,-0.049217,0.344054,0.731914,0.347826,-0.167214,-0.174808,...,0.192444,-0.377179,0.013838,-0.673426,0.370864,-0.128295,-0.052699,0.090457,-0.082435,-0.068107
3,0,-1.597376,-1.625360,1.778601,-0.191016,0.446325,0.465961,-1.339627,-0.009581,-0.062989,...,0.109490,0.390584,-1.189817,-1.083773,0.560616,-0.102875,-0.086089,-0.112172,-0.731119,-0.682985
4,0,-0.111253,-0.279130,-0.979746,0.382140,1.924608,0.593979,-1.683671,0.649551,0.434533,...,-1.385290,-0.147852,-0.842174,0.209532,-1.092224,-0.031822,-0.746799,-0.340772,-0.585002,-0.093938


In [ ]:
df_ps.loc[200:210].compute()

In [ ]:
df_ps.groupby(by = ['Metadata_Source','Metadata_Batch','Metadata_Plate'])

# Neural Net

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
from torchinfo import summary
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=180, profile=None, sci_mode=None)
## Set visible GPU device 
##----------------------------------------------
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

device

device(type='cuda', index=0)

In [77]:
@torch.no_grad()
def validation(val_steps=50):
    loss = 0
    for i in range(val_steps):
        ix = torch.randint(0, val_X.shape[0], (batch_size,))
        Xv, Yv = torch.Tensor(val_X[ix]).to(device), torch.Tensor(val_y[ix]).to(device) # batch X,Y
        logits = model(Xv)
        loss += F.mse_loss(logits, Yv)
    loss /= val_steps
    return loss 

# import torchmetrics 
# evaluate the loss
@torch.no_grad() # this decorator disables gradient tracking inside pytorch
def split_loss(split):
    """
    compute loss for data split passed (training, validation, or test data)
    """
#     from torch.torcheval.metrics import R2Score    
    from torchmetrics.regression import R2Score, PearsonCorrCoef
    x_numpy,y_numpy = {
    'train': (train_X, train_y),
    'val'  : (val_X  , val_y),
    'test' : (test_X , test_y),
    }[split]
    x = torch.Tensor(x_numpy).to(device)
    y = torch.Tensor(y_numpy).to(device) 
    logits = model(x)
#     print(f" size of logits: {logits.shape}   size of y: {y.shape}")
    mse_loss = F.mse_loss(logits, y)
    r2score = R2Score().to(device)
    pearson = PearsonCorrCoef(num_outputs=1).to(device)
    r2_loss = r2score(logits, y) 
    pearson_loss= pearson(logits.view(-1), y.view(-1))
    print(f"\n {split:5s} data:   MSE loss: {mse_loss.item():10.4f}    R2 Score: {r2_loss.item():.5f}     Pearson Coeff. {pearson_loss:.4f}")
    
@torch.no_grad()
def calc_loss(x,y):
    logits = model(x)
    loss = F.mse_loss(logits, y)
    print(y[:20].T)
    print(logits[:20].T)
    print(f"Calculated loss:  {loss.item():5e}")
    

In [78]:
try:
    del model
except:
    pass

In [79]:
# hierarchical network
#  nn.Linear(n_hidden_2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
# del model

n_input    = 1471 # the dimensionality of the character embedding vectors
n_hidden_1 = 128 # the number of neurons in the hidden layer of the MLP
n_hidden_2 = 64 # the number of neurons in the hidden layer of the MLP
model = nn.Sequential(
    nn.Linear(n_input , n_hidden_1, bias=False), 
    nn.BatchNorm1d(n_hidden_1), 
    nn.Tanh(),
    nn.Linear(n_hidden_1, n_hidden_2, bias=False), 
    nn.BatchNorm1d(n_hidden_2), 
    nn.Tanh(),
    nn.Linear(n_hidden_2, 1),
)
model.to(device)

Sequential(
  (0): Linear(in_features=1471, out_features=128, bias=False)
  (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Tanh()
  (3): Linear(in_features=128, out_features=64, bias=False)
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): Tanh()
  (6): Linear(in_features=64, out_features=1, bias=True)
)

In [80]:
# print(type(model))
# print(type(model[-1]))
model[-1].__dict__
model[-1].weight.shape
model[-1].bias.shape
model[-1].weight
# parameter init
with torch.no_grad():
    model[-1].weight *= 0.1 # last layer make less confident
model[-1].weight

{'training': True,
 '_parameters': OrderedDict([('weight', Parameter containing:
               tensor([[ 0.0637,  0.0153,  0.0330, -0.1125,  0.1100, -0.0182, -0.0332,  0.0810,  0.0660, -0.0937, -0.0979,  0.0797,  0.0832, -0.1095, -0.0805, -0.0389, -0.1062,  0.0174, -0.0321,
                         0.0919, -0.0040,  0.0223,  0.1120,  0.0962,  0.1033,  0.0091, -0.1141, -0.0094, -0.0209,  0.0051,  0.0118,  0.1204, -0.0448,  0.0317, -0.0241,  0.0817,  0.0655, -0.1019,
                         0.0340, -0.0249,  0.0001, -0.1076, -0.0778,  0.0504,  0.0431,  0.0688, -0.0387,  0.0350, -0.0902,  0.0019, -0.0464, -0.1084, -0.0634, -0.0594,  0.0267,  0.0912,  0.0635,
                        -0.0772,  0.0750,  0.0389, -0.0762, -0.1133, -0.0507, -0.0159]], device='cuda:0', requires_grad=True)),
              ('bias',
               Parameter containing:
               tensor([0.0593], device='cuda:0', requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_bac

torch.Size([1, 64])

torch.Size([1])

Parameter containing:
tensor([[ 0.0637,  0.0153,  0.0330, -0.1125,  0.1100, -0.0182, -0.0332,  0.0810,  0.0660, -0.0937, -0.0979,  0.0797,  0.0832, -0.1095, -0.0805, -0.0389, -0.1062,  0.0174, -0.0321,
          0.0919, -0.0040,  0.0223,  0.1120,  0.0962,  0.1033,  0.0091, -0.1141, -0.0094, -0.0209,  0.0051,  0.0118,  0.1204, -0.0448,  0.0317, -0.0241,  0.0817,  0.0655, -0.1019,
          0.0340, -0.0249,  0.0001, -0.1076, -0.0778,  0.0504,  0.0431,  0.0688, -0.0387,  0.0350, -0.0902,  0.0019, -0.0464, -0.1084, -0.0634, -0.0594,  0.0267,  0.0912,  0.0635,
         -0.0772,  0.0750,  0.0389, -0.0762, -0.1133, -0.0507, -0.0159]], device='cuda:0', requires_grad=True)

Parameter containing:
tensor([[ 6.3716e-03,  1.5296e-03,  3.3035e-03, -1.1251e-02,  1.1000e-02, -1.8202e-03, -3.3164e-03,  8.1021e-03,  6.6003e-03, -9.3675e-03, -9.7949e-03,  7.9747e-03,  8.3154e-03,
         -1.0947e-02, -8.0478e-03, -3.8921e-03, -1.0619e-02,  1.7369e-03, -3.2101e-03,  9.1863e-03, -3.9974e-04,  2.2270e-03,  1.1204e-02,  9.6206e-03,  1.0326e-02,  9.0521e-04,
         -1.1410e-02, -9.3739e-04, -2.0935e-03,  5.1032e-04,  1.1772e-03,  1.2045e-02, -4.4783e-03,  3.1650e-03, -2.4083e-03,  8.1689e-03,  6.5466e-03, -1.0188e-02,  3.3989e-03,
         -2.4879e-03,  1.2091e-05, -1.0759e-02, -7.7799e-03,  5.0376e-03,  4.3122e-03,  6.8800e-03, -3.8743e-03,  3.5046e-03, -9.0206e-03,  1.8770e-04, -4.6394e-03, -1.0839e-02,
         -6.3418e-03, -5.9408e-03,  2.6716e-03,  9.1247e-03,  6.3474e-03, -7.7236e-03,  7.5003e-03,  3.8908e-03, -7.6186e-03, -1.1329e-02, -5.0721e-03, -1.5879e-03]],
       device='cuda:0', requires_grad=True)

In [88]:
parameters = model.parameters()
ttl_nelements = 0
for p in parameters:
    print(f"Parm shape: {str(p.shape):35s}    # elements: {p.nelement():8d}    Required gradient calc: {p.requires_grad}")
    ttl_nelements += p.nelement()
print(ttl_nelements)

print(f"Total num of parameters: {sum(p.nelement() for p in model.parameters())}") # number of parameters in total
for p in parameters:
    p.requires_grad = True

col_names = [ "input_size", "output_size", "num_params", "params_percent", "mult_adds", "trainable"]
             # "kernel_size"
summary_input_size = (20 * cellpainting_args['sample_size'], 1471)
summary(model, 
        verbose = 2,
        input_size=summary_input_size, col_names = col_names)

# summary(model, 
#         input_size = (1,1032),
#         verbose =2, 
#         col_names = ["input_size", "output_size", "num_params", "params_percent",  "mult_adds","trainable"],
#         col_width=16,
#         row_settings=["var_names"],);

Parm shape: torch.Size([128, 1471])                # elements:   188288    Required gradient calc: True
Parm shape: torch.Size([128])                      # elements:      128    Required gradient calc: True
Parm shape: torch.Size([128])                      # elements:      128    Required gradient calc: True
Parm shape: torch.Size([64, 128])                  # elements:     8192    Required gradient calc: True
Parm shape: torch.Size([64])                       # elements:       64    Required gradient calc: True
Parm shape: torch.Size([64])                       # elements:       64    Required gradient calc: True
Parm shape: torch.Size([1, 64])                    # elements:       64    Required gradient calc: True
Parm shape: torch.Size([1])                        # elements:        1    Required gradient calc: True
196929
Total num of parameters: 196929


NameError: name 'cellpainting_args' is not defined

In [ ]:
torch.manual_seed(42); # seed rng for reproducibility

In [ ]:
start_step = 0
end_step = 200000


In [ ]:
# create a PyTorch optimizer
init_LR = 1e-3
curr_LR = init_LR
step_size = 100000
start_step = 0
end_step  = 200000
batch_size = 64
lossi = []
lossv = []

optimizer = torch.optim.AdamW(model.parameters(), lr=init_LR)

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = step_size, gamma=0.1, last_epoch= -1 , verbose=False)

In [ ]:
model.train();
for i in range(start_step, end_step):
  
    # minibatch construct
    ix = torch.randint(0, train_X.shape[0], (batch_size,))
    Xb, Yb = torch.Tensor(train_X[ix]).to(device), torch.Tensor(train_y[ix]).to(device) # batch X,Y
    
    # forward pass
    logits = model(Xb)
    loss_mse = F.mse_loss(logits, Yb)

    optimizer.zero_grad(set_to_none=True)
    loss_mse.backward()

    optimizer.step()

    lossi.append(loss_mse.log10().item())
    
    scheduler.step()

    # track stats
    if i % 1000 == 0:
        val_loss_mse = validation()
        lossv.append(val_loss_mse.log10().item())
        print(f" {datetime.now().strftime('%X.%f')} | batch: {i:7d}/{end_step:7d} | training loss: {loss_mse.item():11.6f} | validation loss: {val_loss_mse.item():11.6f}") 
    
        if i % 50000 == 0:
            if curr_LR != optimizer.param_groups[0]['lr']:
                curr_LR = optimizer.param_groups[0]['lr']
                print(f" ===> learning rate adjusted to {curr_LR}")        
            model.eval();
            print(f"**{i//50000} iterations**")
            split_loss('train')
            split_loss('val')
            split_loss('test')
            print()
            model.train();

## End of training loop
        
print(f" start_step : {start_step}     end_step: {end_step}    i: {i}")    

model.eval();
split_loss('train')
split_loss('val')
split_loss('test')
model.train();



In [ ]:
# start_step = end_step
# end_step += 50000

In [ ]:
print(f" start_step : {start_step}     end_step: {end_step}     end_step: {end_step - start_step}   i: {i}      learning rate: {optimizer.param_groups[0]['lr']}")    

In [ ]:
# for g in optimizer.param_groups:
#     g['lr']=1e-4
#     print(g)

In [ ]:
print(len(lossi), len(lossv));
# print(lossi[0], lossi[-1])
# lossi.pop()
# print(len(lossi), len(lossv));
# print(lossi[0], lossi[-1])
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1));
plt.plot(torch.tensor(lossv).view(-1));


In [ ]:
https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics:~:text=The%20r2_score%20function%20computes,score%20of%200.0.

In [ ]:
# put layers into eval mode (needed for batchnorm especially)
model.eval();
split_loss('train')
split_loss('val')
split_loss('test')
model.train();

**training using TPSA**

     train data:        MSE loss:   125.6541    R2 Score: 0.88529     Pearson Coeff. 0.9419

     val   data:        MSE loss:   290.9020    R2 Score: 0.67287     Pearson Coeff. 0.8236

     test  data:        MSE loss:   384.7767    R2 Score: 0.67663     Pearson Coeff. 0.8230
      
      

### MinMax TPSA 128 x 64 , no BatchNorm 

**MinMax TPSA 128 x 64 , no BatchNorm**

**50k iterations**

     train data:        MSE loss:     0.0001    R2 Score: 0.79670     Pearson Coeff. 0.8926

     val   data:        MSE loss:     0.0001    R2 Score: 0.68855     Pearson Coeff. 0.8333

     test  data:        MSE loss:     0.0002    R2 Score: 0.66196     Pearson Coeff. 0.8145


**100k iterations**

     train data:        MSE loss:     0.0001    R2 Score: 0.88567     Pearson Coeff. 0.9418

     val   data:        MSE loss:     0.0001    R2 Score: 0.70376     Pearson Coeff. 0.8402

     test  data:        MSE loss:     0.0002    R2 Score: 0.70878     Pearson Coeff. 0.8433
      
      
**150k iterations**

     train data:        MSE loss:     0.0001    R2 Score: 0.88960     Pearson Coeff. 0.9438

     val   data:        MSE loss:     0.0001    R2 Score: 0.70221     Pearson Coeff. 0.8394

     test  data:        MSE loss:     0.0002    R2 Score: 0.70636     Pearson Coeff. 0.8420
      
      
**200k iterations**

    train data:        MSE loss:     0.0001    R2 Score: 0.89114     Pearson Coeff. 0.9441

     val   data:       MSE loss:     0.0001    R2 Score: 0.70325     Pearson Coeff. 0.8394

     test  data:       MSE loss:     0.0002    R2 Score: 0.70603     Pearson Coeff. 0.8416
      

**250k iterations**      
      
     train data:        MSE loss:     0.0001    R2 Score: 0.89125     Pearson Coeff. 0.9440

     val   data:        MSE loss:     0.0001    R2 Score: 0.70360     Pearson Coeff. 0.8398

     test  data:        MSE loss:     0.0002    R2 Score: 0.70624     Pearson Coeff. 0.8417  
      

### MinMax TPSA 128 x 64 , With BatchNorm 

**MinMax TPSA 128 x 64 , with BatchNorm**

**50k iterations**

     train data:    MSE loss:     0.0001    R2 Score: 0.80766     Pearson Coeff. 0.8994

     val   data:    MSE loss:     0.0001    R2 Score: 0.66401     Pearson Coeff. 0.8181

     test  data:    MSE loss:     0.0002    R2 Score: 0.68471     Pearson Coeff. 0.8312


**100k iterations**


     train data:    MSE loss:     0.0001    R2 Score: 0.85963     Pearson Coeff. 0.9295

     val   data:    MSE loss:     0.0001    R2 Score: 0.68456     Pearson Coeff. 0.8308

     test  data:    MSE loss:     0.0002    R2 Score: 0.69034     Pearson Coeff. 0.8358      
      
      
**150k iterations**


     train data:    MSE loss:     0.0001    R2 Score: 0.87261     Pearson Coeff. 0.9345

     val   data:    MSE loss:     0.0001    R2 Score: 0.69643     Pearson Coeff. 0.8354

     test  data:    MSE loss:     0.0002    R2 Score: 0.69435     Pearson Coeff. 0.8362

      
**200k iterations**


     train data:    MSE loss:     0.0001    R2 Score: 0.87318     Pearson Coeff. 0.9345

     val   data:    MSE loss:     0.0001    R2 Score: 0.69366     Pearson Coeff. 0.8337

     test  data:    MSE loss:     0.0002    R2 Score: 0.69301     Pearson Coeff. 0.8363      
     

**250k iterations**      
      
     train data:    MSE loss:     0.0001    R2 Score: 0.87323     Pearson Coeff. 0.9346

     val   data:    MSE loss:     0.0001    R2 Score: 0.69520     Pearson Coeff. 0.8343

     test  data:    MSE loss:     0.0002    R2 Score: 0.69483     Pearson Coeff. 0.8368
      

In [ ]:
from sklearn.metrics import get_scorer_names
for i in get_scorer_names() :
    if "error" in i:
        print(i)

## Evaluation on Test dataset

In [ ]:
model.eval()
# minibatch construct
ix = torch.randint(0, train_X.shape[0], (batch_size,))
Xb, Yb = torch.Tensor(train_X[ix]).to(device), torch.Tensor(train_y[ix]).to(device) # batch X,Y

print(ix)
logits = model(Xb)
with torch.no_grad():
    a,b = logits.cpu().numpy().squeeze(), Yb.cpu().numpy().squeeze()
    print(a.shape, b.shape)
    for i, j in zip(logits.cpu().numpy(), Yb.cpu().numpy()):
        print(f" {i[0]:.5f}     {j[0]:.5f}")

# EDA - Generate Data Profile

In [ ]:
# profile = ProfileReport(df_labels, title="Profiling Report")
# profile_report = df_labels.profile(html={"style": {"full_width": True}})
# profile.to_file("./output/example.html")

### profile widgets

In [ ]:
# profile.to_widgets()

### profile notebook iframe

In [ ]:
# profile.to_notebook_iframe()

## Create label file `all_y1` from `df_tpsa[TPSA]`

In [ ]:
# del tpsa_norm, tpsa_numpy,tpsa_numpy_1, normalizer

###  Normalize data 

In [ ]:
# tpsa_numpy =df_labels['log10TPSA'].to_numpy().reshape(-1,1)
tpsa_numpy =df_labels['TPSA'].to_numpy().reshape(-1,1)


print(f" {tpsa_numpy.shape}  min:  {tpsa_numpy.min()},   max: {tpsa_numpy.max()},  mean: {tpsa_numpy.mean():.4f}, std dev: {tpsa_numpy.std():.4f}")
print("           ",tpsa_numpy[:10].T)

# normalize function 
print(f"\n Using normalize function")
print(f" --------------------------")
normalizer = Normalizer()
tpsa_norm  = normalizer.fit_transform(tpsa_numpy)
print(f" Parameters: {normalizer.get_params(deep=True)}")
# # all_y = normalizer.transform(tpsa_numpy)
print(f" {tpsa_norm.shape}  min:  {tpsa_norm.min()},   max: {tpsa_norm.max():.4f},  mean: {tpsa_norm.mean():.4f}, std dev: {tpsa_norm.std():.4f}")
print(f" First 10 elements : {tpsa_norm[:10].T}")      

print(f"\n Using MaxAbsScaler ")
print(f" --------------------------")
scaler  = MaxAbsScaler()
print(f" type of scaler: {type(scaler)}  ")
tpsa_maxabs  = scaler.fit_transform(tpsa_numpy)
print(f" Parameters: {scaler.get_params(deep=True)}")
# # all_y = normalizer.transform(tpsa_numpy)
print(f" {tpsa_maxabs.shape}  min:  {tpsa_maxabs.min()},   max: {tpsa_maxabs.max():.4f},  mean: {tpsa_maxabs.mean():.4f}, std dev: {tpsa_maxabs.std():.4f}")
print(f"  First 10 elements : {tpsa_maxabs[:10].T}")      

print(f"\n Using MinMaxScaler ")
print(f" --------------------------")
scaler  = MinMaxScaler(feature_range=(-1,1))
print(f" type of scaler: {type(scaler)}")
print(f" Parameters: {scaler.get_params(deep=True)}")
tpsa_minmax  = scaler.fit_transform(tpsa_numpy)
print(f" {tpsa_minmax.shape}  min:  {tpsa_minmax.min()},   max: {tpsa_minmax.max():.4f},  mean: {tpsa_minmax.mean():.4f}, std dev: {tpsa_minmax.std():.4f}")
print(f" First 10 elements : {tpsa_minmax[:10].T}")      
      

In [ ]:
# all_y = tpsa_numpy.copy()
all_y = tpsa_minmax.copy()

In [ ]:
print(f" {all_y.shape} -   min:  {all_y.min()}       max: {all_y.max():.4f}      mean: {all_y.mean():.4f}      std dev: {all_y.std():.4f}")
print(f" First 10 elements : {all_y[:10].T}")


## Read fingerprint features file `all_X`

In [ ]:
print(f" compound Pharmacophore Dense Zipped File : {compoundPharmacophoreDenseZipFile}")

In [ ]:
all_X = np.load(compoundPharmacophoreDenseZipFile)['dense_mat']
print(type(all_X), all_X.shape, all_X.dtype)
print(all_X[:5, :25])